The architecture is::

conv1-->relu-->conv2-->relu-->conv3-->relu-->pool-->

conv4-->relu-->conv5-->relu-->conv6-->relu-->pool-->

conv7-->relu-->conv8-->relu-->conv9-->relu-->pool-->

fully_connected_1-->relu-->fully_connected_2-->relu-->fully_connected_3 **[OUTPUT]**

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
testloader= torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)
classes=('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [2]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 11 Layers : 8 conv layers and 3 fully connected layers !
        self.conv1 = nn.Conv2d(3, 8, 3,padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 12, 3,padding=1)
        self.conv2_bn = nn.BatchNorm2d(12)
        self.conv3 = nn.Conv2d(12,20, 3,padding=1)
        self.conv3_bn = nn.BatchNorm2d(20)
        self.conv4 = nn.Conv2d(20,24, 3,padding=1)
        self.conv4_bn = nn.BatchNorm2d(24)
        self.conv5 = nn.Conv2d(24,32, 3,padding=1)
        self.conv5_bn = nn.BatchNorm2d(32)
        self.conv6 = nn.Conv2d(32,48, 3,padding=1)
        self.conv6_bn = nn.BatchNorm2d(48)
        self.conv7 = nn.Conv2d(48,64, 3,padding=1)
        self.conv7_bn = nn.BatchNorm2d(64)
        self.conv8 = nn.Conv2d(64,72, 3,padding=1)
        self.conv8_bn = nn.BatchNorm2d(72)
        self.conv9 = nn.Conv2d(72,80, 3,padding=1)
        self.conv9_bn = nn.BatchNorm2d(80)
        self.fc1 = nn.Linear(80*4*4, 120)
        self.fc1_bn = nn.BatchNorm2d(120)
        self.fc2 = nn.Linear(120, 84)
        self.fc2_bn = nn.BatchNorm2d(84)
        self.fc3 = nn.Linear(84, 10)

        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2_bn((self.conv2(x))))
        x = self.pool(F.relu(self.conv3_bn((self.conv3(x)))))
        x = F.relu(self.conv4_bn((self.conv4(x))))
        x = F.relu(self.conv5_bn((self.conv5(x))))
        x = self.pool(F.relu(self.conv6_bn((self.conv6(x)))))
        x = F.relu(self.conv7_bn((self.conv7(x))))
        x = F.relu(self.conv8_bn((self.conv8(x))))
        x = self.pool(F.relu(self.conv9_bn((self.conv9(x)))))

        x = x.view(-1, 80*4*4)
        x = F.relu(self.fc1_bn(self.fc1(x)))
        x = F.relu(self.fc2_bn(self.fc2(x)))
        x = self.fc3(x)
        return x

In [3]:
# Function to create an instance of the model on CUDA
def new_net(lrate,mom):

    net = Net().cuda()

    # net=Net()

    lossvsiter=[]

    # To see if the model is on CUDA or not !
    if (next(net.parameters()).is_cuda) :
        print("The model is on CUDA")
    else :
        print("The model is on CPU")

    # Import the optimizers 
    import torch.optim as optim

    # Declare a loss function
    criterion = nn.CrossEntropyLoss()

    # Declare an optimizer
    optimizer = optim.SGD(net.parameters(),lr=lrate,momentum=mom)

    #No of iterations !
    iterations = 25


    for epoch in range(iterations):  # loop over the dataset multiple times

        # Reset the loss for the current epoch !
        running_loss = 0.0

        # Loop over all the mini-batches therea are 12500 mini batches of size 4 each !
        for i, data in enumerate(trainloader, 0):
            # get the inputs
            inputs, labels = data

            # wrap them in Variable & if possible make them cuda tensors
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())

            # zero the parameter gradients for the current epoch
            optimizer.zero_grad()

            # forward + backward + optimize

            # forward
            outputs = net(inputs)
            loss = criterion(outputs, labels)

            # Calculate gradients of whatever variable set to req_gardients = True
            loss.backward()

            # Take one step of the gradient descent for this epoch ! 
            optimizer.step()

            # print statistics
            running_loss += loss.data[0]
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[Epoch :: %d, Mini Batch :: %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                lossvsiter.append(running_loss / 2000)
                running_loss = 0.0


    print('Finished Training')
    return lossvsiter,net

In [4]:
learnin_rate_sample=[0.0005,0.0007,0.001,0.0001,]
weight_decay_smaple=[1e-6,1e-7,1e-5,1e-4,1e-3,1e-2]

In [5]:
lossvsiter_crazy_architecture_with_batch_norm,model_crazy_architecture_with_batch_norm=new_net(0.01,0.9)

RuntimeError: cuda runtime error (30) : unknown error at /pytorch/torch/lib/THC/THCGeneral.c:70

In [ ]:
import pickle

with open("./results/lossvsiter_crazy_architecture_with_batch_norm.pkl","wb") as f:
    pickle.dump(lossvsiter_crazy_architecture_with_batch_norm,f)
    
with open("./results/model_crazy_architecture_with_batch_norm.pkl","wb") as f:
    pickle.dump(model_crazy_architecture_with_batch_norm,f)

In [ ]:
def test_accuracy(model):
    correct = 0
    total = 0

    for data in testloader:
        images, labels = data
        images=images.cuda()
        labels=labels.cuda()
        try:
            outputs = model(Variable(images))
        except RuntimeError as re:
            print(outputs.is_cuda)
            print(str(re))
            sys.exit()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
        accuracy_percentage= 100 * correct / total

    print('Accuracy of the network on the 10000 test images: %d' % (accuracy_percentage))
    print("The network predicted correct for %s"%(correct))
    return accuracy_percentage,correct

In [ ]:
def train_accuracy(model):
    net = model
    total=0
    correct=0
    for i, data in enumerate(trainloader, 0):
        images, labels = data
        images=images.cuda()
        labels=labels.cuda()
        try:
            outputs = net(Variable(images))
        except RuntimeError as re:
            print(outputs.is_cuda)
            print(str(re))
            sys.exit()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
        if (i%1000) == 0:
            print(i)

    print('Accuracy of the network on the 50000 trained images: %d %%' % (
        100 * correct / total))

In [ ]:
test_accuracy(model_crazy_architecture_with_batch_norm)

In [ ]:
train_accuracy(model_crazy_architecture_with_batch_norm)

In [ ]:
net=model_crazy_architecture_with_batch_norm
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
for data in testloader:
    images, labels = data
    images=images.cuda()
    labels=labels.cuda()
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels).squeeze()
    for i in range(4):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
plt.plot(lossvsiter_crazy_architecture_with_batch_norm)
plt.ylabel('loss')
plt.xlabel('every 2000th mini-batch')
plt.show()

In [ ]:
# Lets save the model !!

In [ ]:
torch.save(model_crazy_architecture_with_batch_norm.state_dict(),"./models/model_crazy_architecture_with_batch_norm")

In [ ]:
# It seems the loss will further decrease with more iterations !

In [ ]:
net=model_crazy_architecture_with_batch_norm
lossvsiter=lossvsiter_crazy_architecture_with_batch_norm
# To see if the model is on CUDA or not !
if (next(net.parameters()).is_cuda) :
    print("The model is on CUDA")
else :
    print("The model is on CPU")

# Import the optimizers 
import torch.optim as optim

# Declare a loss function
criterion = nn.CrossEntropyLoss()

# Declare an optimizer
optimizer = optim.Adam(net.parameters(),lr=0.0005,weight_decay=1e-7)

#No of iterations !
iterations = 25


for epoch in range(iterations):  # loop over the dataset multiple times

    # Reset the loss for the current epoch !
    running_loss = 0.0

    # Loop over all the mini-batches therea are 12500 mini batches of size 4 each !
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable & if possible make them cuda tensors
        inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())

        # zero the parameter gradients for the current epoch
        optimizer.zero_grad()

        # forward + backward + optimize

        # forward
        outputs = net(inputs)
        loss = criterion(outputs, labels)

        # Calculate gradients of whatever variable set to req_gardients = True
        loss.backward()

        # Take one step of the gradient descent for this epoch ! 
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[Epoch :: %d, Mini Batch :: %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            lossvsiter.append(running_loss / 2000)
            running_loss = 0.0


print('Finished Training')

In [ ]:
# Lets see the result ::

In [ ]:
test_accuracy(model_crazy_architecture_with_batch_norm)

In [ ]:
train_accuracy(model_crazy_architecture_with_batch_norm)

In [ ]:
torch.save(model_crazy_architecture_with_batch_norm.state_dict(),"./models/model_crazy_architecture_with_batch_norm-50-epochs")

In [ ]:
# Lets run it again for 25 more ... yay !!!!!!!

In [ ]:
model_crazy_architecture_with_batch_norm.load_state_dict(torch.load("./models/model_crazy_architecture_with_batch_norm-50-epochs"))

In [ ]:
model_crazy_architecture_with_batch_norm.eval()

In [ ]:
net=model_crazy_architecture_with_batch_norm
lossvsiter=lossvsiter_crazy_architecture_with_batch_norm
# To see if the model is on CUDA or not !
if (next(net.parameters()).is_cuda) :
    print("The model is on CUDA")
else :
    print("The model is on CPU")

# Import the optimizers 
import torch.optim as optim

# Declare a loss function
criterion = nn.CrossEntropyLoss()

# Declare an optimizer
optimizer = optim.Adam(net.parameters(),lr=0.0005,weight_decay=1e-7)

#No of iterations !
iterations = 25


for epoch in range(iterations):  # loop over the dataset multiple times

    # Reset the loss for the current epoch !
    running_loss = 0.0

    # Loop over all the mini-batches therea are 12500 mini batches of size 4 each !
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable & if possible make them cuda tensors
        inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())

        # zero the parameter gradients for the current epoch
        optimizer.zero_grad()

        # forward + backward + optimize

        # forward
        outputs = net(inputs)
        loss = criterion(outputs, labels)

        # Calculate gradients of whatever variable set to req_gardients = True
        loss.backward()

        # Take one step of the gradient descent for this epoch ! 
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[Epoch :: %d, Mini Batch :: %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            lossvsiter.append(running_loss / 2000)
            running_loss = 0.0


print('Finished Training')

In [ ]:
test_accuracy(model_crazy_architecture_with_batch_norm)

In [ ]:
torch.save(model_crazy_architecture_with_batch_norm.state_dict(),"./models/model_crazy_architecture_with_batch_norm-75-epochs")

In [ ]:
train_accuracy(model_crazy_architecture_with_batch_norm)

In [ ]:
# Lets get in a century

In [ ]:
net=model_crazy_architecture_with_batch_norm
lossvsiter=lossvsiter_crazy_architecture_with_batch_norm
# To see if the model is on CUDA or not !
if (next(net.parameters()).is_cuda) :
    print("The model is on CUDA")
else :
    print("The model is on CPU")

# Import the optimizers 
import torch.optim as optim

# Declare a loss function
criterion = nn.CrossEntropyLoss()

# Declare an optimizer
optimizer = optim.Adam(net.parameters(),lr=0.0005,weight_decay=1e-7)

#No of iterations !
iterations = 25


for epoch in range(iterations):  # loop over the dataset multiple times

    # Reset the loss for the current epoch !
    running_loss = 0.0

    # Loop over all the mini-batches therea are 12500 mini batches of size 4 each !
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable & if possible make them cuda tensors
        inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())

        # zero the parameter gradients for the current epoch
        optimizer.zero_grad()

        # forward + backward + optimize

        # forward
        outputs = net(inputs)
        loss = criterion(outputs, labels)

        # Calculate gradients of whatever variable set to req_gardients = True
        loss.backward()

        # Take one step of the gradient descent for this epoch ! 
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[Epoch :: %d, Mini Batch :: %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            lossvsiter.append(running_loss / 2000)
            running_loss = 0.0


print('Finished Training')

In [ ]:
test_accuracy(model_crazy_architecture_with_batch_norm)

In [ ]:
train_accuracy(model_crazy_architecture_with_batch_norm)

In [ ]:
torch.save(model_crazy_architecture_with_batch_norm.state_dict(),"./models/model_crazy_architecture_with_batch_norm-100-epochs")

In [ ]:
model_crazy_architecture_with_batch_norm.load_state_dict(torch.load("./models/model_crazy_architecture_with_batch_norm-75-epochs"))

In [ ]:
model_crazy_architecture_with_batch_norm.eval()

In [ ]:
test_accuracy(model_crazy_architecture_with_batch_norm)